<a href="https://colab.research.google.com/github/PQ-Trung/GAN/blob/master/GAN_book.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
from numpy import ones, zeros, expand_dims
from numpy.random import randn, randint
from matplotlib import pyplot
import tensorflow as tf
from tensorflow.keras.layers import Dense, LeakyReLU, Flatten 
from tensorflow.keras.layers import Conv2DTranspose, Reshape, Activation
from tensorflow.keras.layers import Conv2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.models import Sequential
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import plot_model
from tensorflow.keras.initializers import RandomNormal

from tensorflow.keras import backend

In [0]:
from google.colab import drive
drive.mount('/gdrive')

In [0]:
cd "/content/drive/My Drive/GANs"

[Errno 2] No such file or directory: '/content/drive/My Drive/GANs'
/content


In [0]:
def wasserstein_loss(y_true, y_pred):
    y_true = tf.convert_to_tensor(y_true,dtype=np.float32)
    y_pred = tf.convert_to_tensor(y_pred,dtype=np.float32)
    m = backend.mean(y_true * y_pred)
    return m

In [0]:
def discriminator_model(input_shape = (32,32,3)):
    init = RandomNormal(stddev = 0.02)
    model = Sequential() # khởi tạo model
    # 32*32*32
    model.add(Conv2D(32, (3,3), padding = 'same',
                     input_shape = input_shape,
                     kernel_initializer = init))
    # model.add(BatchNormalization(momentum=0.00008))
    model.add(LeakyReLU(alpha = 0.2))
    model.add(Dropout(0.25))
    # 16*16*128
    model.add(Conv2D(64, (3,3), padding = 'same',
                     strides = (2,2),kernel_initializer = init))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha = 0.2))
    model.add(Dropout(0.25))
    # # 8*8*128
    model.add(Conv2D(128, (3,3), padding = 'same',
                     strides = (2,2),kernel_initializer = init))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha = 0.2))
    model.add(Dropout(0.25))
    # 4*4*128
    model.add(Conv2D(256, (3,3), padding = 'same',
                     strides = (2,2),kernel_initializer = init))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha = 0.2))
    model.add(Dropout(0.25))
    # Output
    model.add(Flatten())
    # model.add(Dropout(0.25))
    model.add(Dense(1, activation = 'sigmoid'))
    # compile model
    opt = Adam(lr = 0.0003, beta_1=0.5)
    # opt = RMSprop(lr=0.00005)
    model.compile(loss = 'binary_crossentropy',
                  optimizer = opt, metrics = ['accuracy'])
    # model.compile(loss = wasserstein_loss,
    #               optimizer = opt, metrics = ['accuracy'])
    return model

In [0]:
def generator_model(latent_dim):
    model = Sequential() # khởi tạo model
    n_nodes = 256*4*4
    init = RandomNormal(stddev = 0.02)
    # Fc1
    model.add(Dense(n_nodes, input_dim = latent_dim, activation = 'relu', kernel_initializer = init))
    model.add(BatchNormalization())
    # model.add(LeakyReLU(alpha = 0.2))
    model.add(Reshape((4,4,256)))
    # 8*8*128
    model.add(Conv2DTranspose(128, (3,3),
                              strides = (2,2),
                              padding = 'same',
                              kernel_initializer = init
                              , activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    # model.add(LeakyReLU(0.2))
    # 16*16*128
    model.add(Conv2DTranspose(128, (3,3),
                              strides = (2,2),
                              padding = 'same',
                              kernel_initializer = init
                              , activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    # model.add(LeakyReLU(0.2))
    # 32*32*128
    model.add(Conv2DTranspose(128, (3,3),
                              strides = (2,2),
                              padding = 'same', 
                              kernel_initializer = init,
                              activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    # model.add(LeakyReLU(0.2))
    # output 32*32*3 in [-1,1] act = "tanh"
    model.add(Conv2D(3,(3,3), activation = 'tanh',
                     padding = 'same'))
    return model

In [0]:
# def define_discriminator(in_shape=(32,32,3)):
# 	model = Sequential()
# 	# normal
# 	model.add(Conv2D(64, (3,3), padding='same', input_shape=in_shape))
# 	model.add(LeakyReLU(alpha=0.2))
# 	# downsample
# 	model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
# 	model.add(LeakyReLU(alpha=0.2))
# 	# downsample
# 	model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
# 	model.add(LeakyReLU(alpha=0.2))
# 	# downsample
# 	model.add(Conv2D(256, (3,3), strides=(2,2), padding='same'))
# 	model.add(LeakyReLU(alpha=0.2))
# 	# classifier
# 	model.add(Flatten())
# 	model.add(Dropout(0.4))
# 	model.add(Dense(1, activation='sigmoid'))
# 	# compile model
# 	opt = Adam(lr=0.0002, beta_1=0.5)
# 	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
# 	return model

# # define the standalone generator model
# def define_generator(latent_dim):
# 	model = Sequential()
# 	# foundation for 4x4 image
# 	n_nodes = 256 * 4 * 4
# 	model.add(Dense(n_nodes, input_dim=latent_dim))
# 	model.add(LeakyReLU(alpha=0.2))
# 	model.add(Reshape((4, 4, 256)))
# 	# upsample to 8x8
# 	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
# 	model.add(LeakyReLU(alpha=0.2))
# 	# upsample to 16x16
# 	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
# 	model.add(LeakyReLU(alpha=0.2))
# 	# upsample to 32x32
# 	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
# 	model.add(LeakyReLU(alpha=0.2))
# 	# output layer
# 	model.add(Conv2D(3, (3,3), activation='tanh', padding='same'))
# 	return model

In [0]:
def gan_model(g_model, d_model):
    # Không update weights của disc.model trong
    # quá trình train Gan-> để update weight của Gen.model
    d_model.trainable = False
    # Khởi tạo model z > Gen > Disc > real(1) or fake(0)
    model = Sequential()
    model.add(g_model)
    model.add(d_model)
    
    opt = Adam(lr = 0.0002, beta_1 = 0.5)
    model.compile(loss = 'binary_crossentropy', 
                  optimizer = opt, metrics = ['accuracy'])
    # opt = RMSprop(lr=0.00005)
    # model.compile(loss = wasserstein_loss,
    #               optimizer = opt, metrics = ['accuracy'])
    return model

In [0]:
def load_real_samples():
    # load data from dataset
    (trainX, _),(_,_) = cifar10.load_data()
    # chuyển dtype về dạng float
    X = trainX.astype('float32')
    # chuẩn hóa data [0,255] -> [-1,1]
    X = (X - 127.5) / 127.5
    return X

def generate_real_samples(dataset, n_samples):
    # chọn n_samples ở vị trí ngẫu nhiên trong dataset
    ix = randint(0, dataset.shape[0], n_samples)
    # n ảnh X real từ dataset
    X = dataset[ix]
    # y = 1 do X là ảnh thật. 1->[0.7,1.3] better
    y = ones((n_samples, 1)) + np.random.random_sample((n_samples,1))*0.6-0.3
    return X, y

def generate_latent_points(latent_dim, n_samples):
    ## Tạo n vector ngẫu nhiên z theo phân phối chuẩn > uniform
    # z_input = np.random.uniform(0, 1,latent_dim*n_samples)
    z_input = randn(n_samples * latent_dim)
    z_input = z_input.reshape(n_samples, latent_dim)
    return z_input

def generate_fake_samples(g_model, latent_dim, n_samples):
    # Tạo n ảnh X fake từ z thông qua Gen.model
    z_input = generate_latent_points(latent_dim, n_samples)
    X = g_model.predict(z_input)
    # y = 0 do X là ảnh giả.
    y = np.zeros((n_samples, 1)) + 0.3
    return X, y

In [0]:
def save_plot(examples, epoch, n = 7):
    # Chuẩn hóa data về [0,1]
    examples = (examples + 1)/2.0
    # vẽ 7*7 =49 ảnh ngẫu nhiên
    for i in range(n*n):
        pyplot.subplot(n,n,i+1)
        pyplot.axis('off')
        pyplot.imshow(examples[i])
    filename = 'generated_plot_e%03d.png' % (epoch+1)
    pyplot.savefig(filename)
    pyplot.close()

In [0]:
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=150):
	# Chuẩn bị data real
	X_real, y_real = generate_real_samples(dataset, n_samples)
	# tính acc trên data real
	_, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
	# chuẩn bị data fake
	x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
	# tính acc trên data fake
	_, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
	# Độ chính xác của Disc. model
	print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
	# Vẽ hình từ data fake
	save_plot(x_fake, epoch)
	# Save model
	filename = 'generator_model_%03d.h5' % (epoch+1)
	g_model.save(filename)

In [0]:
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=150, n_batch = 256):
	bat_per_epo = int(dataset.shape[0] / n_batch) # số batch cho mỗi epoch
	half_batch = int(n_batch / 2) # số lượng ảnh trong  1 minibatch

	for i in range(n_epochs):
		for j in range(bat_per_epo):
			# Lấy ngẫu nhiên half_batch ảnh thật
			X_real, y_real = generate_real_samples(dataset, half_batch)
			# train disc. model -> loss 1
			d_loss1, _ = d_model.train_on_batch(X_real, y_real)
			# Tạo half_batch ảnh giả
			X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
			# train disc. model -> loss 2
			d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)

			# X, y = np.vstack((X_real, X_fake)), np.vstack((y_real, y_fake))
			# d_loss, _ = d_model.train_on_batch(X, y)
   
			# Tạo n_batch vector latent
			X_gan = generate_latent_points(latent_dim, n_batch)
			# tạo y = 1 để đánh lừa disc.
			y_gan = ones((n_batch, 1))
			# train Gen model thông qua Gan.model
			g_loss, _ = gan_model.train_on_batch(X_gan, y_gan)
			# Sai số ở những batch chia hết cho 100 
			if (j+1)% 10 == 0:
				print('>%d, %d/%d, d1=%.3f, d2=%.3f, g=%.3f' %(i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
		# tính độ chính xác ở những epoch chia hết cho 10
		if (i+1) % 10 == 0:
			summarize_performance(i, g_model, d_model, dataset, latent_dim)

In [0]:
# size of the latent space
latent_dim = 100
# create the discriminator
d_model = discriminator_model()
# create the generator
g_model = generator_model(latent_dim)
# create the gan
gan_model = gan_model(g_model, d_model)
# load image data
dataset = load_real_samples()

In [0]:
# train model
train(g_model, d_model, gan_model, dataset, latent_dim)

>1, 10/195, d1=0.194, d2=0.720, g=0.389
>1, 20/195, d1=0.068, d2=0.650, g=0.125
>1, 30/195, d1=0.267, d2=0.637, g=0.025
>1, 40/195, d1=-0.265, d2=0.611, g=0.007
>1, 50/195, d1=-0.223, d2=0.613, g=0.003
>1, 60/195, d1=-1.091, d2=0.613, g=1.429
>1, 70/195, d1=0.008, d2=0.627, g=0.757
>1, 80/195, d1=-0.018, d2=0.614, g=0.000
>1, 90/195, d1=-0.077, d2=0.615, g=0.000
>1, 100/195, d1=-1.048, d2=0.615, g=0.000
>1, 110/195, d1=-1.061, d2=0.612, g=0.000
>1, 120/195, d1=-4.805, d2=0.631, g=0.000
>1, 130/195, d1=-0.545, d2=0.636, g=0.216
>1, 140/195, d1=2.435, d2=0.620, g=1.412
>1, 150/195, d1=-0.705, d2=0.612, g=2.007
>1, 160/195, d1=0.470, d2=0.643, g=1.100
>1, 170/195, d1=0.449, d2=1.415, g=0.203
>1, 180/195, d1=0.219, d2=0.824, g=0.592
>1, 190/195, d1=0.187, d2=0.780, g=0.684
>2, 10/195, d1=0.320, d2=0.757, g=0.668
>2, 20/195, d1=0.270, d2=0.700, g=0.867
>2, 30/195, d1=0.119, d2=0.688, g=0.948
>2, 40/195, d1=0.113, d2=0.705, g=0.940
>2, 50/195, d1=0.526, d2=0.676, g=0.948
>2, 60/195, d1=0.267